# NCF Cornac

In [46]:
import pandas as pd
import cornac
from cornac.eval_methods import RatioSplit
from cornac.hyperopt import Continuous, Discrete, RandomSearch
from tqdm import tqdm
import numpy as np
import tensorflow as tf

In [47]:
df = pd.read_csv('../data/ncf_data.csv')
df

,p_id,u_id,u_rate
0,f239,batoo2000,5.0
1,f239,Woongs Lee,5.0
2,f239,박우석,5.0
3,f239,EOS,5.0
4,f239,ㅎㅈㅊ,4.0
...,...,...,...
43014,572,쥬,NaN
43015,595,쥬,NaN
43016,213,쥬,NaN
43017,685,쥬,NaN


In [48]:
df.rename(columns = {'p_id' : 'itemID', 'u_id' : 'userID', 'u_rate' : 'rating'}, inplace = True)

In [49]:
ls=[]
for i in range (0,len(df)):
    new_tuple = (df['userID'][i], df['itemID'][i], df['rating'][i])
    ls.append(new_tuple)

In [50]:
ls

[('batoo2000', 'f239', 5.0),
 ('Woongs Lee', 'f239', 5.0),
 ('박우석', 'f239', 5.0),
 ('EOS', 'f239', 5.0),
 ('ㅎㅈㅊ', 'f239', 4.0),
 ('한효선', 'b209', 4.0),
 ('백지현', 'e810', 3.0),
 ('uni', 'e810', 1.0),
 ('Hannah-Gahee U♥', 'a211', 5.0),
 ('김세미', 'a211', 5.0),
 ('쑨꿍', 'a211', 5.0),
 ('제주벼', 'a211', 2.0),
 ('혜진', 'a211', 5.0),
 ('L', 'a211', 2.0),
 ('작은배', 'a211', 1.0),
 ('jw', 'a211', 1.0),
 ('정원', 'a211', 5.0),
 ('꿀벌', 'a211', 1.0),
 ('바지우', 'a211', 1.0),
 ('fjhndklhvnkl', 'a211', 5.0),
 ('eodeoddl', 'a211', 5.0),
 ('나상준', 'a211', 5.0),
 ('문수', 'a211', 5.0),
 ('ㅎㅎ', 'a211', 5.0),
 ('김진영', 'a211', 5.0),
 ('이동훈', 'a211', 5.0),
 ('서수진', 'a211', 1.0),
 ('안영준', 'a211', 1.0),
 ('ryu', 'a211', 1.0),
 ('볼링마니아', 'a211', 1.0),
 ('시닝', 'a211', 1.0),
 ('zjffltmxj', 'a211', 1.0),
 ('영원관세법인', 'a211', 1.0),
 ('이정근', 'a211', 5.0),
 ('카카카', 'a211', 5.0),
 ('비호', 'a211', 1.0),
 ('자의식', 'a211', 3.0),
 ('쥴리', 'b44', 3.0),
 ('2020년 화이팅!', 'b44', 5.0),
 ('DAAS', 'b44', 5.0),
 ('김가현', 'f20', 5.0),
 ('태찌', 'f20', 

In [51]:
ncf = cornac.models.recommender.Recommender('NeuMF')
ncf.early_stop

<bound method Recommender.early_stop of <cornac.models.recommender.Recommender object at 0x644b4a588>>

In [52]:
import cornac
from cornac.eval_methods import RatioSplit

TOP_K = 30

# Define an evaluation method to split feedback into train and test sets
ratio_split = RatioSplit(
    data=ls,
    test_size=0.20,
    rating_threshold=1.0,
    seed=123,
    exclude_unknowns=True,
    verbose=True,
)

neumf = cornac.models.NeuMF(
    num_factors=8,
    layers=[64, 32, 16, 8],
    act_fn="tanh",
    learner="adam",
    num_epochs=200,
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
    early_stopping = {'min_delta':0, 'patience':3}
)


# Instantiate evaluation metrics
ndcg = cornac.metrics.NDCG(k=TOP_K)
pre = cornac.metrics.Precision(k=TOP_K)
rec = cornac.metrics.Recall(k=TOP_K)
fm = cornac.metrics.FMeasure(k=TOP_K)

# RandomSearch
rs_neumf = RandomSearch(
    model = neumf,
    space=[
        Discrete("num_epochs", [50, 100, 150, 200]),
        Discrete("num_factors", [4, 8]),
        Discrete("batch_size", [128, 256, 512]),
        Continuous("lr", 0.001, 0.01)
    ],
    metric = fm,
    eval_method = ratio_split
)


# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=ratio_split,
    models=[neumf],
    metrics=[ndcg, pre, rec, fm],
).run()

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 17464
Number of items = 1181
Number of ratings = 33140
Max rating = nan
Min rating = nan
Global mean = nan
---
Test data:
Number of users = 3380
Number of items = 398
Number of ratings = 5420
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 17464
Total items = 1181

[NeuMF] Training started!


/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/cornac/data/dataset.py:361: UserWarning: 1275 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/cornac/data/dataset.py:361: UserWarning: 219 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)




[NeuMF] Evaluation started!




TEST:
...
      |  F1@30 | NDCG@30 | Precision@30 | Recall@30 | Train (s) | Test (s)
----- + ------ + ------- + ------------ + --------- + --------- + --------
NeuMF | 0.0419 |  0.1679 |       0.0224 |    0.4576 | 8625.7324 |   3.2755



In [ ]:
len(neumf.score(0))